In [1]:
import datetime
import pandas as pd
import time
import requests
import os
import numpy as np

In [2]:
# TODOs:

# PNL reporting/collecting relevant statistics for visualizations from our simulations

# Longer time horizons. Only weight according to n most recent time steps

# Benchmarks: 
## - Buy and hold market index (Dow Jones)
## - Constantly Rebalanced Portfolio (maintain equal weights of each asset at all times)

# Non-fractional shares

# Contexts

# Other trading strategies
## Fundamentals-based strategies (need to get more data for this)



### Simulation runner

In [21]:
path_to_data = "data/djia_20000101_20171101/"
start_date = "2001-01-01"
stdevs = {}
for f in os.listdir(path_to_data):
    df = pd.read_csv(path_to_data+f)
    df = df[df["date"] < start_date]
    df["return"] = (df["adj_close"] - df["adj_close"].shift(1))/df["adj_close"].shift(1)
    stdevs[f] = df["return"].std()

assets = [f.split(".")[0].upper() for f in sorted(stdevs, key=stdevs.get, reverse=True)[:5]]
files

['AAPL', 'INTC', 'CSCO', 'HD', 'GS']

In [33]:

class SimulationEnv(object):
    
    def __init__(self, wealth, assets, 
                 path_to_data, start_date, end_date,
                 agent_type, expert_type, max_assets=50, reinvest=False):

        self.init_wealth = wealth
        self.wealth = wealth
        self.assets = assets
        self.agent_type = agent_type
        self.start_date = start_date
        self.expert_type = expert_type
        self.path_to_data = path_to_data
        self.reinvest = reinvest
        self.max_assets = max_assets
        ## TODO: 
        ## - specify start and end date for simulation
        ## - collect statistics on simulation results
        
        return
    
    
    ## To be called before running a simulation - initialize experts, agents, and initial position, etc.

    def setup_params(self, agent_args={}, expert_args={}):
        
        stdevs = {}
        for f in os.listdir(self.path_to_data):
            df = pd.read_csv(self.path_to_data+f)
            df = df[df["date"] < self.start_date]
            df["return"] = (df["adj_close"] - df["adj_close"].shift(1))/df["adj_close"].shift(1)
            stdevs[f] = df["return"].std()

        self.assets = [f.split(".")[0].upper() for f in sorted(stdevs, key=stdevs.get, reverse=True)[:self.max_assets]]
        
        self.experts = [self.expert_type(a, self.path_to_data, **expert_args) for a in self.assets]
        self.agent = self.agent_type(self.experts, **agent_args)
        self.positions = [weight * self.wealth for weight in self.agent.weights]
        
        ## TODO: 
        # do various other stuff here like for select for high-volatility stocks or something
        # exception handling
        # Need to make sure data files are in sync
        
        return
        
    ## Run simulation
    def run(self):
        ## Warmup period: 
        ## i.e. for strategies involving moving average indicators, wait until we have enough data to calculate MA
        for expert in self.agent.experts:
            expert.warmup()
        
        ## Simulation: Go until data iterators reach the end
        while True:
            try:
                ## Update experts with last period's rewards
                for i, expert in enumerate(self.agent.experts):
                    expert.update()
                    if self.reinvest:# below is to redistribute unallocated money
                        if expert.pick == False:
                            extra_money = self.positions[i]
                            self.positions[i] = 0
                            distribute = np.greater(self.positions, 0)
                            if not np.any(distribute): # this is to avoid divide by zero if nothing is invested
                                #print("all experts say not to buy")
                                self.positions = [weight * self.wealth for weight in self.agent.weights]
                            else:
                                distribute_extra_money = distribute * (extra_money/ np.sum(distribute))
                                self.positions = np.add(self.positions, distribute_extra_money)
                            assert(abs(np.sum(self.positions)-self.wealth) <= 1)# make sure all money is accounted for
                      
                ## Update agent accordingly (i.e. for Hedge, update weights according to each expert's reward in the last period)
                self.agent.update()
                
                ## Update position with returns from last period
                self.positions = self.positions * (1 + self.agent.returns)
                self.wealth = np.sum(self.positions)
                
                print self.agent.weights
                ## Rebalance according to new, updated weights
                ## TODO: only allow non-fractional shares to be purchased (?)
                if np.sum(self.agent.weights) == 1:
                    self.positions = [weight * self.wealth for weight in self.agent.weights]
            except StopIteration:
                break



### Base classes

Base classes for agents and experts to be implemented by us

In [25]:
from abc import ABCMeta, abstractmethod

class Agent(object):
    __metaclass__ =ABCMeta
    
    @abstractmethod
    def __init__(self):
        pass
    
    @abstractmethod
    def update(self):
        pass
    
    @abstractmethod
    def act(self):
        pass
    
class Expert(object):
    __metaclass__=ABCMeta
    
    @abstractmethod
    def __init__(self, reward_data):
        pass
    
    @abstractmethod
    def update(self):
        pass
    
class Context(object):
    __metaclass__=ABCMeta
    
    @abstractmethod
    def __init__(self, context_data):
        pass
    
    @abstractmethod
    def observe(self):
        pass

### Agents

Implementations of different online portfolio selection algorithms
* Exponential Gradient
* Exponential Gradient (recent history only)

In [26]:
from queue import Queue
from collections import deque

## EG. (http://www.cis.upenn.edu/~mkearns/finread/helmbold98line.pdf)
class EG(Agent):
    
    ## Initialize with a set of experts
    def __init__(self, experts, eta):
        self.eta = eta
        self.experts = experts
        self.weights = np.ones(len(self.experts))/len(self.experts)
        self.rewards = None
        self.returns = None
        return
    
    ## Update the agent's rewards and its weights for each expert
    def update(self):
        self.rewards = np.asarray([e.reward for e in self.experts])
        self.returns = np.asarray([e.returns for e in self.experts])
        multipliers = np.exp(self.eta * self.rewards/np.sum(self.weights * self.rewards))
        self.weights = (self.weights * multipliers)/ np.sum(self.weights * multipliers)
        self.weights = np.nan_to_num(self.weights)

        return
    
    def act(self):
        return self.weights


class EGRecent(Agent):
    
    ## Initialize with a set of experts
    def __init__(self, experts, eta, n):
        self.eta = eta
        self.experts = experts
        #self.weights = Queue(maxsize=n)
        self.weights = np.ones(len(self.experts))/len(self.experts)
        self.rewards = deque(maxlen=n)
        self.returns = None
        self.t = 0
        self.n = n
        #self.rewards = None
        #self.returns = None
        return
    
    ## Update the agent's rewards and its weights for each expert
    def update(self):
           
        self.rewards.append(np.asarray([e.reward for e in self.experts]))
        self.returns = np.asarray([e.returns for e in self.experts])
        
        self.weights = np.ones(len(self.experts))/len(self.experts)
        #print self.t
        #print len(self.rewards)
        for rewards in self.rewards:
            rewards = np.asarray(rewards)
            multipliers = np.exp(self.eta * rewards/np.sum(self.weights * rewards))
            self.weights = (self.weights * multipliers)/ np.sum(self.weights * multipliers)
            self.weights = np.nan_to_num(self.weights)
        self.t += 1
        #print self.t
        
        return
    
    def act(self):
        return self.weights
    
      
        

More agents to serve as naive/benchmark portfolio allocation algorithms:
* Equal-weighted buy and hold
* Market-weighted buy and hold
* Constantly rebalanced portfolio


In [27]:
## Some other examples of agents to use as benchmarks 

class NaiveBuyHold(Agent):
    
    def __init__(self, experts):
        self.experts = experts
        self.weights = np.ones(len(self.experts))/len(self.experts)
        self.rewards = None
        return
    
    def update(self):
        self.rewards = np.asarray([e.reward for e in self.experts])
        self.returns = self.rewards
        return
    
    def act(self):
        return self.weights
    
## TODO
class WeightedBuyHold(Agent):
    
    def __init__(self):
        return
    
    def update(self):
        return
    
    def act(self):
        return
    

## TODO
class ConstantRebalancer(Agent):
    
    def __init__(self):
        return
    
    def update(self):
        return
    
    def act(self):
        return


### Experts

* Dummy
* Mean Reversion
* Momentum

In [28]:

## Dummy expert that always pick the same asset
class Dummy(Expert):
    
    ## Expert has a reward associated with its pick
    def __init__(self, name, path_to_data):
        self.reward = 0.
        self.pick = True ## Might not be necessary
        self.data = pd.read_csv(path_to_data + name + ".csv", iterator=True, chunksize=1)
        self.last_price = float(self.data.get_chunk(1)["adj_close"])
        return
    
    ## Expert updates its reward 
    def update(self):
        current_price = float(self.data.get_chunk(1)["adj_close"])
        self.reward = (current_price - self.last_price)/self.last_price
        self.returns = self.reward
        self.last_price = current_price
        return
    
    def warmup(self):
        pass
    

class MeanReversion(Expert):
    
    def __init__(self, name, path_to_data, window_size, threshold):
        self.reward = 0.
        self.pick = False
        self.data = pd.read_csv(path_to_data + name + ".csv", iterator=True, chunksize=1)
        self.last_price = float(self.data.get_chunk(1)["adj_close"])
        self.window_size = window_size
        self.avg = 0.0
        self.std = 0.0
        self.threshold = threshold
        self.last_n_prices = Queue(maxsize=10)
        self.returns = 0.
        return
    
    def warmup(self):
        n = 1
        while n <= self.window_size:
            self.last_n_prices.put(self.last_price)
            self.last_price = float(self.data.get_chunk(1)["adj_close"])
            n += 1
        return
        
    def update(self):
        _ = self.last_n_prices.get()
            
        self.last_n_prices.put(self.last_price)
        self.avg = np.mean(list(self.last_n_prices.queue))
        self.std = np.std(list(self.last_n_prices.queue))
        
        current_price = float(self.data.get_chunk(1)["adj_close"])

        ## If self.pick is True, we bought the stock and our reward is whatever the return was in the last period
        if self.pick:
            self.reward = (current_price - self.last_price)/self.last_price
            self.returns = self.reward
        else:
            self.reward = -(current_price - self.last_price)/self.last_price
            self.returns = 0
        self.last_price = current_price

        if self.last_price <= self.avg - self.threshold * self.std:
            self.pick = True
        else:
            self.pick = False

        return
    
    
class Momentum(Expert):
    
    def __init__(self, name, path_to_data, window_size, threshold):
        self.reward = 0.
        self.pick = False
        self.data = pd.read_csv(path_to_data + name + ".csv", iterator=True, chunksize=1)
        self.last_price = float(self.data.get_chunk(1)["adj_close"])
        self.window_size = window_size
        self.avg = 0.0
        self.std = 0.0
        self.threshold = threshold
        self.last_n_prices = Queue(maxsize=10)
        self.returns = 0.
        return
    
    def warmup(self):
        n = 1
        while n <= self.window_size:
            self.last_n_prices.put(self.last_price)
            self.last_price = float(self.data.get_chunk(1)["adj_close"])
            n += 1
        return
        
    def update(self):
        _ = self.last_n_prices.get()
            
        self.last_n_prices.put(self.last_price)
        self.avg = np.mean(list(self.last_n_prices.queue))
        self.std = np.std(list(self.last_n_prices.queue))
        
        current_price = float(self.data.get_chunk(1)["adj_close"])

        ## If self.pick is True, we bought the stock and our reward is whatever the return was in the last period
        if self.pick:
            self.reward = (current_price - self.last_price)/self.last_price
            self.returns = self.reward
        else:
            self.reward = -(current_price - self.last_price)/self.last_price
            self.returns = 0
        self.last_price = current_price

        if self.last_price >= self.avg - self.threshold * self.std:
            self.pick = True
        else:
            self.pick = False

        return
  

### Contexts

In [29]:
  
class VolContext(Context):
    
    def __init__(self, data_file, threshold):
        self.data = pd.read_csv(data_file, iterator=True, chunksize=1)
        self.threshold = threshold
        self.contexts = ["HighVol", "LowVol"]
        return
    
    # Returns a string giving the name of the context
    def observe(self):
        if float(self.data.get_chunk(1)["adj_close"]) > self.threshold:
            return self.contexts[0]
        else:
            return self.contexts[1]

In [30]:
stocks = ['AAPL', 'AXP', 'BA', 'CAT', 'CSCO', 'CVX', 'DD', 'DIS', 'GE',
 'GS', 'HD', 'IBM', 'INTC', 'JNJ', 'JPM', 'KO', 'MCD', 'MMM', 
'MRK', 'MSFT', 'NKE', 'PFE', 'PG', 'TRV', 'UNH', 'UTX', 'V', 'VZ', 'WMT', 'XOM']

In [31]:
initial_wealth = 100000

In [48]:
## Run a simulation with Hedge, each expert is a proxy for buying a given stock
start = time.time()

s = SimulationEnv(initial_wealth, stocks, "data/djia_20150101_20171101/", "2016-01-01", "2017-11-01", EG, Dummy, max_assets = 5, reinvest=True)
s.setup_params(agent_args = {"eta": -0.01})
s.run()

years = 2. + 11./12
ar = ((s.wealth)/initial_wealth)**(1/years) - 1
end = time.time()

print "Initial wealth: ${}".format(initial_wealth)
print "Final wealth: ${}".format(s.wealth)
print "Annualized return: {}".format(ar)
print "Time: {}s".format(int(end-start))

[ 0.20141116  0.19945638  0.20028099  0.20020624  0.19864522]
[ 0.19898828  0.20131198  0.19877205  0.20223923  0.19868845]
[ 0.19891223  0.20347011  0.19751207  0.20194486  0.19816072]
[ 0.19868047  0.20373748  0.19718595  0.20102178  0.19937432]
[ 0.19909119  0.20195619  0.19658881  0.20324283  0.19912098]
[ 0.19883388  0.2000667   0.2007703   0.20078019  0.19954893]
[ 0.19932421  0.19756783  0.19792569  0.20534721  0.19983506]
[ 0.19730602  0.19821279  0.19870239  0.20556963  0.20020917]
[ 0.19755372  0.19821048  0.20115726  0.20293263  0.20014591]
[ 0.19558051  0.19456449  0.1995457   0.20685607  0.20345323]
[ 0.1968263   0.19373207  0.20120936  0.2030442   0.20518807]
[ 0.20242154  0.19014339  0.20408179  0.20225525  0.20109803]
[ 0.20145825  0.19129816  0.20471929  0.20133048  0.20119381]
[ 0.2037171   0.18904815  0.20270392  0.20454599  0.19998484]
[ 0.20724889  0.18371603  0.20292689  0.2061342   0.19997399]
[ 0.20500383  0.185319    0.20438138  0.20656279  0.198733  ]
[ 0.1966

[ 0.20619267  0.0754943   0.49825739  0.03473154  0.1853241 ]
[ 0.20406908  0.07607602  0.50147135  0.03426974  0.18411381]
[ 0.20927331  0.07625464  0.50201073  0.03458775  0.17787357]
[ 0.21093835  0.07594398  0.49964267  0.03483205  0.17864296]
[ 0.20969957  0.07519328  0.50236071  0.03458246  0.17816397]
[ 0.2099369   0.05725768  0.55982326  0.03331091  0.13967125]
[ 0.34362261  0.06956465  0.44118176  0.02958182  0.11604916]
[ 0.34045502  0.07051136  0.44402536  0.03010591  0.11490235]
[ 0.33957428  0.06122365  0.45237487  0.02957837  0.11724882]
[ 0.34400562  0.05928986  0.45194225  0.02897781  0.11578446]
[ 0.37301929  0.05850594  0.43119339  0.0251391   0.11214227]
[ 0.38236089  0.04393699  0.42256331  0.02912926  0.12200955]
[ 0.37502322  0.04526957  0.4256221   0.02951067  0.12457443]
[ 0.37996728  0.04477659  0.41985055  0.02992977  0.12547583]
[ 0.3812717   0.04462925  0.42012275  0.02966415  0.12431214]
[ 0.3813808   0.04504867  0.42046386  0.02919346  0.12391321]
[ 0.3814

[ 0.70549313  0.02394675  0.21187037  0.02089761  0.03779214]
[ 0.70679504  0.02383432  0.210661    0.02107487  0.03763477]
[ 0.70599292  0.02399881  0.21178764  0.02093443  0.0372862 ]
[ 0.70627625  0.02387395  0.21159713  0.02102575  0.03722691]
[ 0.7087431   0.02348318  0.21045598  0.02064903  0.03666871]
[ 0.70672355  0.02360519  0.21256778  0.02077009  0.03633339]
[ 0.70576031  0.02339316  0.21404332  0.02084897  0.03595423]
[ 0.70551889  0.02324611  0.21434101  0.02088291  0.03601109]
[ 0.70574569  0.02320743  0.21457934  0.02024661  0.03622093]
[ 0.70633566  0.02343531  0.21349104  0.0203609   0.03637709]
[ 0.6978288   0.0232929   0.22121884  0.02060705  0.03705241]
[ 0.69752206  0.02320566  0.22049804  0.02075377  0.03802046]
[ 0.6974682   0.02321208  0.22044146  0.02071276  0.0381655 ]
[ 0.68546419  0.02240771  0.2334739   0.02073666  0.03791754]
[ 0.68136099  0.02277368  0.23452428  0.0211634   0.04017765]
[ 0.71209522  0.02233061  0.20908505  0.01951342  0.0369757 ]
[ 0.6999

[ 0.02569633  0.08010677  0.8728663   0.006171    0.0151596 ]
[ 0.02603299  0.08218747  0.86896245  0.00637074  0.01644635]
[ 0.02655285  0.08058469  0.86930033  0.00686308  0.01669906]
[ 0.02652256  0.08023448  0.86981456  0.00685019  0.0165782 ]
[ 0.02916736  0.08561458  0.85922456  0.00786176  0.01813174]
[ 0.02929734  0.08503485  0.85952585  0.00805665  0.01808531]
[ 0.028514    0.07942822  0.8617196   0.01065646  0.01968173]
[ 0.02826226  0.07922775  0.8625574   0.01035394  0.01959864]
[ 0.0286022   0.07754411  0.86370196  0.01034745  0.01980428]
[ 0.03081268  0.07647609  0.86026902  0.01195816  0.02048405]
[ 0.03104447  0.07692711  0.85931546  0.01207703  0.02063593]
[ 0.03054037  0.07531742  0.86179738  0.0122014   0.02014342]
[ 0.0310573   0.07785688  0.85757739  0.01311596  0.02039247]
[ 0.03046231  0.07608208  0.86164064  0.01232285  0.01949213]
[ 0.03003817  0.0759646   0.86185718  0.01249558  0.01964448]
[ 0.0298084   0.07657909  0.86128936  0.01259864  0.01972451]
[ 0.0299

[  9.99999999e-01   2.44784037e-69   5.89058429e-10   7.27400820e-32
   3.47651926e-90]
[  9.99999999e-01   2.39079924e-69   5.83450678e-10   7.11346816e-32
   3.57996379e-90]
[  9.99999999e-01   2.35371186e-69   5.87761409e-10   7.13569418e-32
   3.55069333e-90]
[  9.99999999e-01   2.35649687e-69   5.88082435e-10   7.08103830e-32
   3.48160756e-90]
[  9.99999999e-01   2.38001490e-69   5.92008269e-10   7.15841896e-32
   3.52580283e-90]
[  9.99999999e-01   2.38996925e-69   5.94750108e-10   7.18956522e-32
   3.50641989e-90]
[  9.99999999e-01   2.38262697e-69   5.64760621e-10   7.18346215e-32
   3.09096547e-90]
[  9.99999999e-01   2.29846944e-69   5.40359624e-10   6.92650179e-32
   3.17665522e-90]
[  9.99999999e-01   2.34224101e-69   5.61264172e-10   6.98567111e-32
   3.30353552e-90]
[  9.99999999e-01   2.26869428e-69   5.76647696e-10   7.08060614e-32
   3.38614003e-90]
[  9.99999999e-01   2.30359842e-69   5.83813099e-10   7.04670836e-32
   3.42644360e-90]
[  9.99999999e-01   2.25040771e-

In [47]:
s.assets

['MSFT', 'CVX']

In [49]:
## Run a simulation with Hedge, each expert is a proxy for buying a given stock
start = time.time()

s = SimulationEnv(initial_wealth, stocks, "data/djia_20000101_20171101/", "2001-01-01", "2017-11-01", EG, Dummy, max_assets = 5)
s.setup_params(agent_args = {"eta": -0.001})
s.run()

years = 17. + 11./12
ar = ((s.wealth)/initial_wealth)**(1/years) - 1
end = time.time()

print "Initial wealth: ${}".format(initial_wealth)
print "Final wealth: ${}".format(s.wealth)
print "Annualized return: {}".format(ar)
print "Time: {}s".format(int(end-start))

[ 0.1999611   0.20006781  0.20004014  0.1999093   0.20002164]
[ 0.19992222  0.1999354   0.20011186  0.19920557  0.20082495]
[ 0.19963944  0.19974767  0.20015976  0.19924711  0.20120603]
[ 0.19961331  0.1997506   0.200078    0.1991689   0.20138919]
[ 0.20002643  0.19939547  0.19983036  0.19942709  0.20132065]
[ 0.19931281  0.20041501  0.19949304  0.19916623  0.20161292]
[ 0.19877322  0.20083131  0.19937934  0.19926506  0.20175107]
[ 0.1982403   0.20104612  0.19942366  0.19934684  0.20194308]
[ 0.19824384  0.20056359  0.19955532  0.19956183  0.20207542]
[ 0.19783783  0.20093416  0.19904135  0.20016763  0.20201903]
[ 0.19739069  0.20164504  0.19917104  0.19977715  0.20201608]
[ 0.2020801   0.19873232  0.1994156   0.19876661  0.20100538]
[ 0.20366949  0.19722646  0.1977889   0.19990504  0.2014101 ]
[ 0.20360135  0.19747523  0.19767448  0.1999093   0.20133964]
[ 0.2032655   0.19741087  0.1976169   0.20041212  0.20129462]
[ 0.20295701  0.19625649  0.19663529  0.20122038  0.20293084]
[ 0.2032

[ 0.17828518  0.29524583  0.19651464  0.18794192  0.14201242]
[ 0.17855351  0.29532574  0.1967111   0.18765251  0.14175714]
[ 0.17852998  0.29536175  0.19662176  0.1878983   0.14158821]
[ 0.17737472  0.29633368  0.19685402  0.18772848  0.1417091 ]
[ 0.17743419  0.29583863  0.19674119  0.18817212  0.14181387]
[ 0.17724072  0.2956395   0.19676761  0.18842163  0.14193052]
[ 0.17698504  0.29562101  0.19671865  0.18862951  0.14204579]
[ 0.17673204  0.29548037  0.19657545  0.18925888  0.14195325]
[ 0.178496    0.29964201  0.20779718  0.18321253  0.13085228]
[ 0.17814305  0.30022842  0.2060399   0.18478167  0.13080696]
[ 0.17837469  0.29933701  0.20684104  0.1847048   0.13074246]
[ 0.17814461  0.29871447  0.20702405  0.18518449  0.13093237]
[ 0.17805301  0.29845498  0.20749917  0.18514932  0.13084352]
[ 0.18254422  0.29656984  0.20552843  0.18599594  0.12936157]
[ 0.18281997  0.29582078  0.20544765  0.18648197  0.12942963]
[ 0.18281675  0.29539564  0.20544733  0.18666843  0.12967184]
[ 0.1824

[ 0.15928659  0.30781771  0.24366466  0.17921011  0.11002094]
[ 0.15932903  0.30786281  0.24342476  0.17934972  0.11003367]
[ 0.15988482  0.3079851   0.24295797  0.17927838  0.10989373]
[ 0.16586787  0.31407339  0.24239334  0.17280008  0.10486532]
[ 0.16599709  0.31396241  0.24236101  0.17273896  0.10494053]
[ 0.16560841  0.31492107  0.24186571  0.17284286  0.10476194]
[ 0.16566159  0.3147823   0.24178127  0.17297986  0.10479499]
[ 0.1655748   0.31426829  0.24180163  0.17328231  0.10507296]
[ 0.16551329  0.31387487  0.24194203  0.17351249  0.10515733]
[ 0.17673919  0.31525471  0.22899242  0.16882914  0.11018453]
[ 0.17640426  0.31527085  0.22867957  0.16938892  0.1102564 ]
[ 0.17634741  0.31527127  0.2288238   0.16932994  0.11022759]
[ 0.17644651  0.31521181  0.22862283  0.16948648  0.11023236]
[ 0.17690984  0.31508198  0.22812587  0.16943226  0.11045005]
[ 0.16000164  0.29999024  0.26939648  0.17687827  0.09373337]
[ 0.16002444  0.30006095  0.26916215  0.17696617  0.09378629]
[ 0.1599

[ 0.12049426  0.30471653  0.28550086  0.18730756  0.1019808 ]
[ 0.12063009  0.30438411  0.28544891  0.18731922  0.10221767]
[ 0.12058485  0.30424438  0.28544169  0.18748707  0.10224202]
[ 0.12071951  0.30407652  0.28644891  0.18663018  0.10212489]
[ 0.12073359  0.30412267  0.28638691  0.18663003  0.10212681]
[ 0.12076166  0.30400082  0.28640632  0.18666544  0.10216576]
[ 0.12085996  0.30422683  0.28541725  0.18743965  0.10205632]
[ 0.11973015  0.30450426  0.28625969  0.18756388  0.10194202]
[ 0.11984794  0.30191311  0.28765583  0.18841387  0.10216926]
[ 0.11989561  0.30071034  0.28769133  0.18983699  0.10186573]
[ 0.11992361  0.30060092  0.28786826  0.18971868  0.10188853]
[ 0.11999206  0.3004614   0.28799822  0.18957533  0.10197299]
[ 0.11989833  0.29963373  0.28907469  0.18999646  0.10139678]
[ 0.11972384  0.29978801  0.2891923   0.18990578  0.10139007]
[ 0.11961544  0.29996415  0.28913165  0.18991085  0.1013779 ]
[ 0.119642    0.29993086  0.28899561  0.18998189  0.10144965]
[ 0.1208

[ 0.10520334  0.33291851  0.2526971   0.20180586  0.10737519]
[ 0.10515983  0.33287702  0.25260846  0.2019639   0.10739079]
[ 0.10570357  0.33359794  0.25140801  0.20145289  0.10783759]
[ 0.10576096  0.33351248  0.2513767   0.20148141  0.10786845]
[ 0.10576668  0.33344833  0.25132894  0.20144827  0.10800778]
[ 0.10555777  0.33320317  0.2515058   0.20165503  0.10807823]
[ 0.10566131  0.33306465  0.25151916  0.20165236  0.10810251]
[ 0.10570102  0.33303506  0.25152119  0.20166343  0.10807929]
[ 0.10570614  0.3329608   0.25154907  0.20164925  0.10813475]
[ 0.10578712  0.33275301  0.25146137  0.2017603   0.1082382 ]
[ 0.10727525  0.33109723  0.25185     0.20111729  0.10866022]
[ 0.10720228  0.33138958  0.25170306  0.20112029  0.10858478]
[ 0.10916054  0.32872941  0.2529982   0.20019109  0.10892076]
[ 0.10917405  0.32874915  0.25288432  0.20023999  0.10895248]
[ 0.10926406  0.32862338  0.2528812   0.20018675  0.10904461]
[ 0.10930127  0.32852349  0.25295711  0.20005029  0.10916783]
[ 0.1093

...

[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  

[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  

[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  

[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  

[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  

In [ ]:
s.agent.weights

In [ ]:
## Run a simulation with Hedge, each expert is a proxy for buying a given stock

s = SimulationEnv(initial_wealth, stocks, "data/djia_20150101_20171101/", "2015-01-01", "2017-11-01", EGRecent, Dummy)
s.setup_params(agent_args = {"eta": -0.01, "n": 100})
s.run()

years = 2. + 11./12
ar = ((s.wealth)/initial_wealth)**(1/years) - 1
end = time.time()

print "Initial wealth: ${}".format(initial_wealth)
print "Final wealth: ${}".format(s.wealth)
print "Annualized return: {}".format(ar)
print "Time: {}s".format(int(end-start))

In [ ]:
## Run a simulation with Hedge, each expert is a proxy for buying a given stock

s = SimulationEnv(initial_wealth, stocks, "data/djia_20000101_20171101/", "2000-01-01", "2017-11-01", EGRecent, Dummy)
s.setup_params(agent_args = {"eta": -0.01, "n": 500})
s.run()

years = 17. + 11./12
ar = ((s.wealth)/initial_wealth)**(1/years) - 1
end = time.time()

print "Initial wealth: ${}".format(initial_wealth)
print "Final wealth: ${}".format(s.wealth)
print "Annualized return: {}".format(ar)
print "Time: {}s".format(int(end-start))

In [ ]:
## Run simulation where we just buy and hold all stocks in equal amounts
start = time.time()
s = SimulationEnv(initial_wealth, stocks, "data/djia_20150101_20171101/", "2015-01-01", "2017-11-01", NaiveBuyHold, Dummy)
s.setup_params(expert_args={})
s.run()

years = 2. + 11./12
ar = ((s.wealth)/initial_wealth)**(1/years) - 1
end = time.time()

print "Initial wealth: ${}".format(initial_wealth)
print "Final wealth: ${}".format(s.wealth)
print "Annualized return: {}".format(ar)
print "Time: {}s".format(int(end-start))



In [ ]:
## Run simulation where we just buy and hold all stocks in equal amounts
start = time.time()
s = SimulationEnv(initial_wealth, stocks, "data/djia_20000101_20171101/", "2000-01-01", "2017-11-01", NaiveBuyHold, Dummy)
s.setup_params(expert_args={})
s.run()

years = 17. + 11./12
ar = ((s.wealth)/initial_wealth)**(1/years) - 1
end = time.time()

print "Initial wealth: ${}".format(initial_wealth)
print "Final wealth: ${}".format(s.wealth)
print "Annualized return: {}".format(ar)
print "Time: {}s".format(int(end-start))

In [ ]:
## Run simulation where we each expert only recommends to buy when the price crosses 0.5 standard deviations below the 10-day moving average. 
## Otherwise doesn't do anything
## Sell position when price gets above 0.5 standard deviations below the 10-day MA

start = time.time()

s = SimulationEnv(initial_wealth, stocks, "data/djia_20150101_20171101/", "2015-01-01", "2017-11-01", EG, MeanReversion)

s.setup_params(
    agent_args={"eta": -0.01},
    expert_args={"window_size": 10, "threshold": .5}
)

s.run()
end = time.time()
years = 2. + 11./12
ar = ((s.wealth)/initial_wealth)**(1/years) - 1

print "Initial wealth: ${}".format(initial_wealth)
print "Final wealth: ${}".format(s.wealth)
print "Annualized return: {}".format(ar)
print "Time: {}s".format(int(end-start))

In [ ]:

start = time.time()

s = SimulationEnv(initial_wealth, stocks, "data/djia_20000101_20171101/", "2000-01-01", "2017-11-01", EG, MeanReversion)

s.setup_params(
    agent_args={"eta": -0.01},
    expert_args={"window_size": 10, "threshold": .5}
)

s.run()
end = time.time()
years = 17. + 11./12
ar = ((s.wealth)/initial_wealth)**(1/years) - 1

print "Initial wealth: ${}".format(initial_wealth)
print "Final wealth: ${}".format(s.wealth)
print "Annualized return: {}".format(ar)
print "Time: {}s".format(int(end-start))

In [ ]:

start = time.time()

s = SimulationEnv(initial_wealth, stocks, "data/djia_20000101_20171101/", "2000-01-01", "2017-11-01", EGRecent, MeanReversion)

s.setup_params(
    agent_args={"eta": -0.01, "n": 1250},
    expert_args={"window_size": 10, "threshold": .5}
)

s.run()
end = time.time()
years = 17. + 11./12
ar = ((s.wealth)/initial_wealth)**(1/years) - 1

print "Initial wealth: ${}".format(initial_wealth)
print "Final wealth: ${}".format(s.wealth)
print "Annualized return: {}".format(ar)
print "Time: {}s".format(int(end-start))

In [ ]:
start = time.time()
s = SimulationEnv(initial_wealth, stocks, "data/djia_20150101_20171101/", "2015-01-01", "2017-11-01", EG, Momentum)

years = 2. + 11./12
s.setup_params(
    agent_args={"eta": -0.01},
    expert_args={"window_size": 10, "threshold": 2.5}
)

s.run()
end = time.time()

ar = ((s.wealth)/initial_wealth)**(1/years) - 1

print "Initial wealth: ${}".format(initial_wealth)
print "Final wealth: ${}".format(s.wealth)
print "Annualized return: {}".format(ar)
print "Time: {}s".format(int(end-start))

In [ ]:
start = time.time()
s = SimulationEnv(initial_wealth, stocks, "data/djia_20000101_20171101/", "2000-01-01", "2017-11-01", EG, Momentum)

years = 17. + 11./12
s.setup_params(
    agent_args={"eta": -0.01},
    expert_args={"window_size": 10, "threshold": 2.5}
)

s.run()
end = time.time()

ar = ((s.wealth)/initial_wealth)**(1/years) - 1

print "Initial wealth: ${}".format(initial_wealth)
print "Final wealth: ${}".format(s.wealth)
print "Annualized return: {}".format(ar)
print "Time: {}s".format(int(end-start))

In [ ]:
start = time.time()
s = SimulationEnv(initial_wealth, stocks, "data/djia_20000101_20171101/", "2000-01-01", "2017-11-01", EGRecent, Momentum)

years = 17. + 11./12
s.setup_params(
    agent_args={"eta": -0.001, "n": 500},
    expert_args={"window_size": 10, "threshold": 2.5}
)

s.run()
end = time.time()

ar = ((s.wealth)/initial_wealth)**(1/years) - 1

print "Initial wealth: ${}".format(initial_wealth)
print "Final wealth: ${}".format(s.wealth)
print "Annualized return: {}".format(ar)
print "Time: {}s".format(int(end-start))